In [ ]:
from gnomad_db.gnomad_db import gnomAD_DB
import pandas as pd
import gzip
from tqdm import tqdm
from joblib import Parallel, delayed

In [ ]:
# initialize database
db = gnomAD_DB("/s/project/variantDatabase/fastStorage/gnomAD/3.1.1")

In [ ]:
# read variants from tsv.gz table in batches
def load_batches(file, batch_size=100_000):
    with gzip.open(file, "rb") as f:
        batch = []
        for line in tqdm(f):
            line = line.decode().rstrip()
            if len(batch) == batch_size:
                batch = pd.DataFrame(batch, columns=db.columns).replace(".", "NULL")
                yield batch
                batch = []

            batch.append(line.split("\t"))
        
        
        if len(batch) != 0:
            batch = pd.DataFrame(batch, columns=db.columns).replace(".", "NULL")
            yield batch
        

In [ ]:
#for batch in load_batches("data/gnomad.genomes.v3.1.1.sites.chr1.tsv.gz", 50000):
#    # insert variants
#    db.insert_variants(batch)

In [ ]:
tables = glob.glob("/data/ouga/home/ag_gagneur/nonchev/code/gnomAD/data/*.tsv.gz")
tables

In [ ]:
# load variants in parallel

In [ ]:
for table in tqdm(tables):
    Parallel(12)(delayed(db.insert_variants)(batch) for batch in load_batches(table))